In [4]:
include("../src/JuTrack.jl")

Main.JuTrack

In [5]:
using .JuTrack
using Enzyme
using Test
using BenchmarkTools

In [1]:
Threads.nthreads()

64

In [31]:
# function fodo_ring(K1, RAD)
#         D1 = DRIFT(name="D1", len=0.34)
#         D2 = DRIFT(name="D2", len=0.59)
#         QF1 = KQUAD(name="QF1", len=0.32, k1=K1, rad=RAD )
#         QD1 = KQUAD(name="QD1", len=0.32, k1=-1.192160, rad=RAD  )
#         BendingAngle = pi/2
#         BD1 = SBEND(name="BD1", len=0.72, angle=BendingAngle/2, e1=BendingAngle/2, e2=0.0 , rad=RAD )
#         BD2 = SBEND(name="BD2", len=0.72, angle=BendingAngle/2, e1=0.0, e2=BendingAngle/2, rad=RAD  )

#         FODO = (QF1, D1, QD1, D1, QF1)
#         BD1_comp = (BD1)
#         BD2_comp = (BD2)
#         D2_comp = (D2)

#         CELL = Vector{AbstractElement}(undef, 8)
#         for i in eachindex(FODO)
#                 CELL[i] = FODO[i]
#         end
#         CELL[length(FODO)+1] = BD1
#         CELL[length(FODO)+2] = D2
#         CELL[length(FODO)+3] = BD2
#                 # CELL[i+172] = M5[i]
        
#         ELIST = Vector{AbstractElement}(undef, 4*length(CELL))
#         for i in eachindex(CELL)
#                 ELIST[i] = CELL[i]
#                 ELIST[i+length(CELL)] = CELL[i]
#                 ELIST[i+2*length(CELL)] = CELL[i]
#                 ELIST[i+3*length(CELL)] = CELL[i]
#         end
#         return ELIST
# end

function create_sbend(BendingAngle)
        SBD = SBEND(name="BD", len=0.72, angle=BendingAngle/2, e1=BendingAngle/2, e2=0.0 , rad=0.0)
        return [SBD]
end        

function sbend_track_mthread(BendingAngle)
        particles = [.001 .0001 .0005 .0002 0.0 0.0] 
        beam = Beam(particles)
        line = create_sbend(BendingAngle)
        plinepass!(line, beam)
        return beam.r
end

function sbend_track(BendingAngle)
        particles = [.001 .0001 .0005 .0002 0.0 0.0] 
        beam = Beam(particles)
        line = create_sbend(BendingAngle)
        linepass!(line, beam)
        return beam.r
end


function generate_report(time_track, time_track_mthread, time_ad, time_ad_mthread)
        
        if time_track[1] == time_track_mthread[1]
                println("Multithreading matches single thread tracking")
        else
                println("Multithreading DOES NOT match single thread tracking")
        end
        
        
        # grad1 = autodiff(Forward, sbend_track, DuplicatedNoNeed, Duplicated(bend_angle, 1.0))
        # grad2 = autodiff(Forward, sbend_track_mthread, DuplicatedNoNeed, Duplicated(bend_angle, 1.0))
        
        if time_ad[1] == time_ad_mthread[1]
                println("Multithreading matches single thread AD")
        else
                println("Multithreading DOES NOT match single thread AD")
        end
        println("")
        println("")
        
        
        
        
        # Benchmarking
        
        
        #Tracking
        println("Tracking")
        println("")
        println("Single Thread Tracking Time: ", time_track[2])
        println("Multi Thread Tracking Time: ", time_track_mthread[2])
        if time_track[2] < time_track_mthread[2]
                println("Single thread tracking is faster by " * string(time_track_mthread[2] - time_track[2]) * " seconds")
        else
                println("Multithread tracking is faster by " * string(time_track[2] - time_track_mthread[2]) * " seconds")
        end
        
        println("")
        println("Single Thread tracking Memory: ", time_track[3] , " bytes")
        println("Multi Thread tracking Memory: ", time_track_mthread[3], " bytes")
        if time_track[2] < time_track_mthread[2]
                println("Single thread tracking uses less memory by " * string(time_track_mthread[3] - time_track[3]) * " bytes")
        else
                println("Multithread tracking uses less memory faster by " * string(time_track[3] - time_track_mthread[3]) * " bytes")
        end
        
        # AD
        println("AD")
        println("")
        println("Single Thread AD Time: ", time_ad[2])
        println("Multi Thread AD Time: ", time_ad_mthread[2])
        if time_ad[2] < time_ad_mthread[2]
                println("Single thread tracking is faster by " * string(time_ad_mthread[2] - time_ad[2]) * " seconds")
        else
                println("Multithread tracking is faster by " * string(time_ad[2] - time_ad_mthread[2]) * " seconds")
        end
        
        println("")
        println("Single Thread AD Memory: ", time_ad[3] , " bytes")
        println("Multi Thread AD Memory: ", time_ad_mthread[3], " bytes")
        if time_ad[2] < time_ad_mthread[2]
                println("Single thread AD uses less memory by " * string(time_ad_mthread[3] - time_ad[3]) * " bytes")
        else
                println("Multithread AD uses less memory faster by " * string(time_ad[3] - time_ad_mthread[3]) * " bytes")
        end
        return
end

generate_report (generic function with 1 method)

One particle Tests:

In [33]:
bend_angle = pi/2

time_track = @timed sbend_track(bend_angle)
time_track_mthread = @timed sbend_track_mthread(bend_angle)

time_ad = @timed autodiff(Forward, sbend_track, DuplicatedNoNeed, Duplicated(bend_angle, 1.0))
time_ad_mthread = @timed autodiff(Forward, sbend_track_mthread, DuplicatedNoNeed, Duplicated(bend_angle, 1.0))

generate_report(time_track, time_track_mthread, time_ad, time_ad_mthread)

Multithreading matches single thread tracking
Multithreading matches single thread AD


Tracking

Single Thread Tracking Time: 7.9199e-5
Multi Thread Tracking Time: 0.033045665
Single thread tracking is faster by 0.032966466 seconds

Single Thread tracking Memory: 4560 bytes
Multi Thread tracking Memory: 54928 bytes
Single thread tracking uses less memory by 50368 bytes
AD

Single Thread AD Time: 5.0525e-5
Multi Thread AD Time: 0.010936073
Single thread tracking is faster by 0.010885548 seconds

Single Thread AD Memory: 9600 bytes
Multi Thread AD Memory: 78208 bytes
Single thread AD uses less memory by 68608 bytes


In [11]:
# println(@benchmark sbend_track(bend_angle))
@benchmark sbend_track_mthread(bend_angle)

BenchmarkTools.Trial: 8103 samples with 1 evaluation.
 Range (min … max):  114.966 μs … 40.742 ms  ┊ GC (min … max): 0.00% … 96.50%
 Time  (median):     158.858 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   611.381 μs ±  2.974 ms  ┊ GC (mean ± σ):  3.07% ±  3.02%

  █▃                                                           ▁
  ███▆▆▅▆▅▅▅▅▅▅▅▅▅▄▁▁▁▄▃▄▄▄▅▄▄▅▄▁▃▃▄▄▁▄▃▃▄▁▃▃▅▃▄▄▄▄▅▃▃▃▄▄▁▄▅▅▄ █
  115 μs        Histogram: log(frequency) by time      19.5 ms <

 Memory estimate: 52.77 KiB, allocs estimate: 397.

In [15]:
@btime sbend_track(bend_angle)

  2.180 μs (36 allocations: 4.45 KiB)


1×6 Matrix{Float64}:
 0.00147904  7.0715e-5  0.000251301  -0.000345415  0.00102709  0.0